In [1]:
import os
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import urllib
from urllib.parse import urlencode, quote_plus
from pandas.io.json import json_normalize

%matplotlib inline

In [13]:
user_ether_address = input("Input your Ethereum Wallet Address: ")

Input your Ethereum Wallet Address:  0x1220e048b0174AD09A2ab8cadE8845f444d44E54


In [3]:
print(user_ether_address)

0x1220e048b0174AD09A2ab8cadE8845f444d44E54


In [5]:
#!NOT USED!#
#This was the old God's Unchained API @ https://github.com/immutable/gods-unchained-api
#I learned a couple days after using this that a new API had replaced it on their new platform Immutable X (IMX)

#get Gods Unchained Metadata Endpoint token ID detail
url = "https://api.godsunchained.com/metadata?collection=0xacb3c6a43d15b907e8433077b6d38ae40936fe2c&token_ids=1456,5423"
querystring = {"page_size":"1"}
headers = {"Accept": "application/json"}
response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

404 page not found



In [6]:
#!NOT USED!#
#Token Trove API - originally was going to be used.  After checking with creator regarding Autentication Token he suggested using the Immutable X marketplace instead

url = "https://api.tokentrove.com/cached"
headers = {"x-api-key" : "Np8BV2d5QR9TSFEr9EvF66FWcJf0wIxy2qBpOH6s"}

response_data = requests.request("GET", url, headers=headers)
#print(json.dumps(response_data, indent=4))
print(response_data.text)

{"message":"Missing Authentication Token"}


In [7]:
#get user inventory  #currently not working API down

#https://market.x.immutable.com/assets/0xacb3c6a43d15b907e8433077b6d38ae40936fe2c?direction=asc&order_by=buy_quantity&page_size=48&sell_token_type=ERC721&status=active
url = f"https://api.x.immutable.com/v1/asset?collection=0xacb3c6a43d15b907e8433077b6d38ae40936fe2c&owner={user_ether_address}&status=imx"
querystring = {"page_size":"1"}
headers = {"Accept": "application/json"}
response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

404 page not found



In [14]:
# Get a list of collections
# Each game or marketplace would likely have its own collection
# 0xacb3c6a43d15b907e8433077b6d38ae40936fe2c is the contract address "seller" of Immutable X Market place

#currently only God's Unchained is on Immutable X

url = "https://api.x.immutable.com/v1/collections"
headers = {"Accept": "application/json"}
response = requests.request("GET", url, headers=headers)

print(response.text)

{"result":[{"address":"0xacb3c6a43d15b907e8433077b6d38ae40936fe2c","name":"Gods Unchained","description":null,"icon_url":null}],"cursor":"","remaining":0}


In [4]:
# Get stark keys for a registered user.  Not used in this program at this time. 
# Stark Key which is a public key defined over a Stark-friendly elliptic curve that is different from the standard Ethereum elliptic curve

url = f"https://api.x.immutable.com/v1/users/{user_ether_address}"
headers = {"Accept": "application/json"}
response = requests.request("GET", url, headers=headers)

print(response.text)

{"accounts":["0x06b9eddbb4ef23041479423b49ac234ad53dc95525672dc78cca0be2652d01bd"]}


In [15]:
#Fetches the WEI balances of the user
url = f"https://api.x.immutable.com/v1/balances/{user_ether_address}"
headers = {"Accept": "application/json"}
response = requests.request("GET", url, headers=headers)

print(response.text)

{"imx":"197970000000000000","preparing_withdrawal":"0","withdrawable":"0"}


In [11]:
#Get a list of collection filters for the card set
#this can be used to filter or search the Immutable X Marketplace
#not actively using these but good reference

url = "https://api.x.immutable.com/v1/collections/0xacb3c6a43d15b907e8433077b6d38ae40936fe2c/filters"
querystring = {"page_size":"1"}
headers = {"Accept": "application/json"}
response = requests.request("GET", url, headers=headers, params=querystring).json()

df_filters = pd.DataFrame.from_dict(response)
print(json.dumps(response, indent=2))

[
  {
    "key": "attack",
    "value": [
      "0",
      "1",
      "13",
      "2",
      "3",
      "4",
      "5",
      "6",
      "7",
      "8",
      "9"
    ],
    "type": "discrete"
  },
  {
    "key": "effect",
    "value": [
      "",
      "Ability: Give one of your creatures without an afterlife, \u201cAfterlife: Summon a 3/2 Experimental Outcome.\u201d",
      "Ability: Target creature goes to sleep. Remove 1 durability from this relic.",
      "Add a random Death card, Nature card, and War card to your hand.",
      "Add four random War creatures to your hand, and give +2/+2 to them.",
      "Add two mana locks.<br>Heal your god for 12.",
      "After a friendly creature attacks, gain 3 favor.",
      "After a friendly creature attacks, give +1/+1 to a friendly creature with the lowest strength.<br>At the end of the turn, remove 1 durability from this relic.",
      "After either player summons a creature, you gain 2 favor.<br>Afterlife: If you have 40 favor<br>or more

In [6]:
#Get a list of cards / assets for a given ETH addresss

#can only pull 100 records at a time so will need to loop through to get full set of cards. 
#IMX provides a remaining count and a cursor to make it easier to iterate through recordsets
#df_GUCards will be the master dataframe
records_remaining_flag = True
response_cursor = ''
df_GUCards = pd.DataFrame()

while (records_remaining_flag == True):
    url = "https://api.x.immutable.com/v1/assets"
    querystring = {"user":user_ether_address,"collection":"0xacb3c6a43d15b907e8433077b6d38ae40936fe2c", "cursor":response_cursor, "page_size":"500" }
    headers = {"Accept": "application/json"}

    #This should pull in a full set of cards that the user owns
    response_data = requests.request("GET", url, headers=headers, params=querystring).json()
    response_cursor = response_data['cursor']
    
    #first data frame record level info for card
    df1 = pd.DataFrame.from_dict(response_data['result'])
    df1 = df1.reset_index()

    #second data frame that contains metadata of the card
    #should move this out of the while loop
    df2 = pd.DataFrame(df1['metadata'].tolist())
    df2 = df2.reset_index()

    #merge dataframs resulting in complete set of cards from a user
    df_merge = pd.merge(df1, df2)
    df_merge.drop(['index'], axis=1, inplace=True)
    df_GUCards = df_GUCards.append(df_merge, ignore_index=True)
    
    if (response_data['remaining'] > 0):
        records_remaining = response_data['remaining']
    else:
        records_remaining_flag = False

#reset index on main dataframe
#I feel like there's a better way to handle indexes in these dataframes
df_GUCards.reset_index(inplace=True)
        
#add amount fields.
df_GUCards['ETH amount'] = ''
df_GUCards['USD amount'] = ''

#print(json.dumps(response_data, indent=4))
#df_GUCards.to_csv('export.csv')
#df.columns 

In [7]:
#get USD price of ether from coingecko
#https://www.coingecko.com/en/api#explore-api

ethusd_url = "https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd"
coingecko_response_data = requests.request("GET", ethusd_url).json()
eth_price = coingecko_response_data['ethereum']['usd']

In [8]:
#iterate through each record/cord of the users set and lookup the price from the Immutable X (IMX) Marketplace
#price returned is the lowest available price listed.  Historical prices are not included but would be nice to have in future versions. 
#for reference: https://api.x.immutable.com/v1/orders?direction=asc&order_by=buy_quantity&page_size=100&sell_token_name=battle%20priest&sell_token_type=ERC721&status=active

for index, row in df_GUCards.iterrows():
    #set paramaters to be passed in as sell_metadata.  Essentially what card are we looking for in the IMX Marketplace
    sellparams = {"proto":[str(df_GUCards['proto'][index])],"quality":[df_GUCards['quality'][index]]}
    sellparams = json.dumps(sellparams)

    #convert sell_metadata from json to encoded format for passing through URL. Need to do this twice.
    sellparams = urllib.parse.quote_plus(sellparams)
    sellparams = urllib.parse.quote_plus(sellparams)

    url = f"https://api.x.immutable.com/v1/orders?direction=asc&order_by=buy_quantity&sell_metadata={sellparams}&sell_token_address=0xacb3c6a43d15b907e8433077b6d38ae40936fe2c&status=active&page_size=1"

    #querystring = {"order_by":"buy_quantity","direction":"asc","status":"active","sell_token_address":"0xacb3c6a43d15b907e8433077b6d38ae40936fe2c", "sell_metadata": sellparams}
    headers = {"Accept": "*/*"}

    IMX_market_response = requests.request("GET", url, headers=headers).json()
    card_amount = IMX_market_response['result'][0]['buy']['data']['quantity']
    
    #assign price of card from IMX marketplace to datafrime as price and pad to 18 digits
    #Price is in ETH but can be converted to USD with coingecko 
    #df_GUCards['price'][index] = IMX_market_response['result'][0]['buy']['data']['quantity']
    df_GUCards.loc[index, 'ETH amount'] = float('.' + (card_amount).zfill(18))
    df_GUCards.loc[index, 'USD amount'] = df_GUCards.loc[index, 'ETH amount'] * eth_price

In [16]:
#df_GUCards['USD Amount'].sum()
total_card_count = df_GUCards['index'].count()
unique_card_count = df_GUCards['proto'].nunique()


most_valuable_card = df_GUCards[df_GUCards['USD amount']==df_GUCards['USD amount'].max()]
most_valuable_name = most_valuable_card.iloc[0]['name']
most_valuable_amount = most_valuable_card.iloc[0]['USD amount']
most_valuable_image = most_valuable_card.iloc[0]['image']

ETH_total = df_GUCards.sum(axis = 0, skipna = True)[22]
USD_total = df_GUCards.sum(axis = 0, skipna = True)[23]

In [10]:
print('****************************************************************************************************************************')
print('*                                                                                                                          *')
print(f'*     Gods Unchained Player!!  You own: {unique_card_count} unique cards and {total_card_count} total cards.                                              *') 
print(f'*     The estimated worth of your set is {ETH_total:.4f} Ether which converts to ${USD_total:.2f} USD!                                      *')
print('*                                                                                                                          *')
print(f'*     Your most valuable card is {most_valuable_name} worth ${most_valuable_amount:.2f} dollars!                                          *')
print(f'*     See this card here: {most_valuable_image}                                                       *')
print('*                                                                                                                          *')
print('*                                                                                                                          *')
print(f'*                                                            Ethereum Wallet @ {user_ether_address}  *')
print('****************************************************************************************************************************')

****************************************************************************************************************************
*                                                                                                                          *
*     Gods Unchained Player!!  You own: 120 unique cards and 295 total cards.                                              *
*     The estimated worth of your set is 0.3575 Ether which converts to $1493.55 USD!                                      *
*                                                                                                                          *
*     Your most valuable card is Neferu, Champion of Death worth $213.05 dollars!                                          *
*     See this card here: https://card.godsunchained.com/?id=828&q=3                                                       *
*                                                                                                                          *


In [11]:
#Export cardset to csv file
df_GUCards.to_csv('export_cardset_05112021.csv')

In [ ]:
#Version 2.0 notes

#iterate through image links and download .png files
#build a database of historical prices so can graph and trend
